In [ ]:
# Original setting

# Zookeeper Server 
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/zookeeper-server-start.sh /Users/hyunseokjung/kafka_2.12-3.3.1/config/zookeeper.properties
# Kafka Server 
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-server-start.sh /Users/hyunseokjung/kafka_2.12-3.3.1/config/server.properties

# kafka connect
# /Users/hyunseokjung/confluent-7.3.0/bin/connect-distributed -daemon /Users/hyunseokjung/confluent-7.3.0/etc/kafka/connect-distributed.properties

# Kafka Topic
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --create --topic recommender-system --partitions 1 --replication-factor 1
# kafka topic list
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --list
# kafka topic delete
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-topics.sh --delete --bootstrap-server 127.0.0.1:9092 --topic recommender-system
# Kafka Producer
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-console-producer.sh --bootstrap-server 127.0.0.1:9092 --topic recommender-system
# Kafka Consumer
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic recommender-system

# csv to kafka topic
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --create --topic ratings --partitions 1 --replication-factor 1
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic ratings < /Users/hyunseokjung/data/ml-latest-small/ratings.csv
# csv consumer
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic ratings --from-beginning

In [ ]:
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --create --topic ratings --partitions 1 --replication-factor 1
# /Users/hyunseokjung/kafka_2.12-3.3.1/bin/kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic ratings < https://cs11003bffd9e29a9a6.blob.core.windows.net/movies-recommender-system/ratings.csv

In [1]:
import findspark
findspark.init()

In [10]:
# import os
# import sys
# spark_path = 'SPARK_HOME=/Users/hyunseokjung/spark-3.3.0'
# os.environ['SPARK_HOME'] = spark_path
# sys.path.insert(0, spark_path + "/bin")
# sys.path.insert(0, spark_path + "/python/pyspark/")
# sys.path.insert(0, spark_path + "/python/lib/pyspark.zip")
# sys.path.insert(0, spark_path + "/python/lib/py4j-0.10.9.5-src.zip")

In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY = '5g'

spark = SparkSession.builder \
    .appName('recommender_system_kafka') \
    .master('local') \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 00:11:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/12 00:11:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/12 00:11:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/12 00:11:33 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [6]:
socketDF = spark.readStream.format("socket") \
    .option("host", "localhost") \
    .option("port", 9093).load()

22/12/12 00:14:49 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [9]:
socketDF.

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
socket

In [11]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
  .option("subscribe", "topic1") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        

In [12]:
df1 = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
    .option("subscribe", "recommender-system") \
    .load()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        

In [31]:
df1.schema

StructType([StructField('key', BinaryType(), True), StructField('value', BinaryType(), True), StructField('topic', StringType(), True), StructField('partition', IntegerType(), True), StructField('offset', LongType(), True), StructField('timestamp', TimestampType(), True), StructField('timestampType', IntegerType(), True)])

In [13]:
spark.read.format('csv') \
    .option('header', 'true') \
    .load('https://github.com/ingu627/BigData/blob/master/data/ratings.csv') \
    .repartition(10)

Py4JJavaError: An error occurred while calling o95.load.
: java.lang.UnsupportedOperationException
	at org.apache.hadoop.fs.http.AbstractHttpFileSystem.listStatus(AbstractHttpFileSystem.java:95)
	at org.apache.hadoop.fs.http.HttpsFileSystem.listStatus(HttpsFileSystem.java:23)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.util.H

In [42]:
df1.writeStream \
    .format("console") \
    .outputMode('append') \
    .option("checkpointLocation", "./tempo_dir/") \
    .start()

22/12/10 23:05:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/12/10 23:05:57 WARN StreamingQueryManager: Stopping existing streaming query [id=aaa924f3-5a24-46e4-ae93-8af27a99c045, runId=81474712-b07f-4a51-9f69-60200f31e3e6], as a new run is being started.
